In [1]:
import openpyxl
from text_processing import readJSON, common_grams, bag, get_gram_id,remove_rows_with_none_elements, strip_text
from features import feat_cnt, constant
from sklearn.linear_model import LinearRegression,LogisticRegression

from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB, ComplementNB,CategoricalNB

from sklearn.decomposition import PCA 
from sklearn.cluster import KMeans 
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd 
from gensim.models import Word2Vec
from sklearn.decomposition import NMF
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\danhj\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\danhj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
def trunc(n, k):
    """
    Return the number truncated to k decimal places, e.g, k = 0 returns only the integer part
    """

    return int(n * 10**k) / 10**k

def measure(y_true, y_pred, k):
    """
    y_pred = binary vector
    y_true = binary vector

    Desc: Computes the accuracy, TP, TN, FP, FN, BER truncated to k decimal places
    """
    
    if len(y_pred) != len(y_true):
        print("Lengths not equal: y_pred={}, y_true={}".format(len(y_pred), len(y_true)))
        return

    TP, TN, FP, FN = 0, 0, 0, 0,
    for i in range(len(y_pred)):
        if y_pred[i] == y_true[i] and y_true[i] == 1:
            TP += 1
        elif y_pred[i] == y_true[i] and y_true[i] == 0:
            TN += 1
        elif y_pred[i] != y_true[i] and y_true[i] == 1:
            FN += 1
        elif y_pred[i] != y_true[i] and y_true[i] == 0:
            FP += 1 
    
    total = TP + TN + FP + FN
    print(total)
    FPR = TP / (TP + FP) if TP + FP > 0 else 0
    FPN = TN / (TN + FN) if TN + FN > 0 else 0
    acc = (TP + TN) / total
    precision = TP / (TP + FP) if TP + FP > 0 else 0
    recall = TP / (TP + FN) if TP + FN > 0 else 0
    f_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    BER = (FPR + FPN) / 2
    print(f_score, acc)
    return (TP, TN, FP, FN, trunc(precision, k), trunc(recall, k), trunc(acc, k), trunc(f_score, k))

def prod(S):
    product = 1
    for x in S:
        product *= x
    return product

In [3]:
# Open the Excel file
excel_file_path = 'Book1.xlsx'
workbook = openpyxl.load_workbook(excel_file_path)

# Select the active sheet (you can also select a specific sheet by name: workbook['Sheet1'])
sheet = workbook.active

# Create empty lists to store data from each column
columns = [[] for _ in range(sheet.max_column)]
counter = 0
# Iterate through each row and store data from each column
for row in sheet.iter_rows(values_only=True):
    counter +=1
    for i, cell_value in enumerate(row):
        
        columns[i].append(cell_value)


# Close the workbook
workbook.close()


In [4]:
def clean_text(text):
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove punctuation and stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    # Stem words
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]
    return ' '.join(tokens)

In [5]:
columns = remove_rows_with_none_elements(columns)
train_data = []
train_data.append(columns[0][0:2587])
train_data.append(columns[1][0:2587])

test_data = []
test_data.append(columns[0][2587:])
test_data.append(columns[1][2587:])

for i in train_data[0]:
    i = clean_text(i)
for i in test_data[0]:
    i = clean_text(i)


In [6]:

def create_dominant_topics(dataset1, dataset2):
    """
    Create dominant topics using NMF factorization on two datasets.

    Args:
        dataset1 (list): List of documents for the first dataset.
        dataset2 (list): List of documents for the second dataset.

    Returns:
        tuple: A tuple containing two lists of dominant topics for each dataset.
    """
    max_length = len(dataset2)
    
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(dataset1)
    tfidf_matrix2 = tfidf_vectorizer.fit_transform(dataset2)
    # NMF factorization
    num_topics = 10
    nmf_model = NMF(n_components=num_topics, random_state=27)
    nmf_model.fit(tfidf_matrix)
    
    dominant_topics = np.argsort(nmf_model.transform(tfidf_matrix), axis=1)[:, ::-1][:, :5]
    
    nmf_model.fit(tfidf_matrix2)
    dominant_topics2 = np.argsort(nmf_model.transform(tfidf_matrix2), axis=1)[:, ::-1][:, :5]
    feature_names = tfidf_vectorizer.get_feature_names_out()
    for topic_idx, topic in enumerate(nmf_model.components_):
        top_words_idx = topic.argsort()[-30:][::-1]
        top_words = [feature_names[i] for i in top_words_idx]
        print(f"Topic {topic_idx + 1}: {', '.join(top_words)}")
    # Print top words for each topic

    res = []
    for i in dominant_topics:
        res.append(i)
    
    res2 = []
    for i in dominant_topics2:
        res2.append(i)  
    
    return res, res2, dominant_topics, dominant_topics2

In [7]:
grouped_words,grouped_words_test,dominate_topics1, dominate_topics2 = create_dominant_topics(train_data[0], test_data[0])


Topic 1: ve, time, like, day, got, week, just, little, ll, going, good, really, today, didn, did, year, new, said, home, know, make, days, food, bit, night, lot, think, went, friends, work
Topic 2: nbsp, therizinosaurus, therizinosaurs, therizinosaur, luis, art, lesotho, skywatch, perez, live, just, subscribing, evolved, ichthyosaurs, world, time, like, shop, blog, clutch, skies, blogged, spring, farm, orthodox, ichthyosaur, did, think, thanks, church
Topic 3: abt, ur, person, da, dat, wat, frnd, guy, best, really, dear, ppl, dont, class, tat, wen, like, hav, gud, lik, ma, friend, friends, say, testimonial, ya, wud, knw, testi, close
Topic 4: new, blog, com, site, camera, great, mso, post, work, music, book, like, photo, use, francisco, san, web, page, photos, www, http, check, health, project, magazine, company, time, color, years, 2010
Topic 5: people, don, just, want, know, love, life, like, think, things, really, make, say, right, thing, feel, need, doing, live, maybe, ve, guy, sur

C:\Users\danhj\AppData\Roaming\Python\Python312\site-packages\sklearn\decomposition\_nmf.py:1770: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


In [8]:

model = LogisticRegression(max_iter=1000)
model_2 = ComplementNB()
y = [1 if d == 'M' else 0 for d in train_data[1]] 
model.fit(grouped_words, y)
model_2.fit(grouped_words, y)

y_pred = model.predict(grouped_words_test)
y_pred2 = model_2.predict(grouped_words_test)


y_test = [1 if d == 'M' else 0 for d in test_data[1]]

grouped_words_regression = "grouped_words_regression: (TP, TN, FP, FN, precision, recall, acc, f_score) = {}".format(measure(y_test, y_pred, 5))
grouped_words_NB = "grouped_words_NB: (TP, TN, FP, FN, precision, recall, acc, f_score) = {}".format(measure(y_test, y_pred2, 5))

639
0.4033970276008493 0.5602503912363067
639
0.5587301587301587 0.564945226917058


In [9]:
print(grouped_words_regression)
print(grouped_words_NB)
print(dominate_topics1)

grouped_words_regression: (TP, TN, FP, FN, precision, recall, acc, f_score) = (95, 263, 72, 209, 0.56886, 0.3125, 0.56025, 0.40339)
grouped_words_NB: (TP, TN, FP, FN, precision, recall, acc, f_score) = (176, 185, 150, 128, 0.53987, 0.57894, 0.56494, 0.55873)
[[2 0 3 1 8]
 [2 8 3 9 7]
 [3 6 2 7 0]
 ...
 [5 3 9 8 7]
 [3 2 8 0 6]
 [8 3 2 0 7]]
